In [ ]:
!pip install fancyimpute
!pip install streamlit
!pip install joblib==1.4.2

In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import joblib
from scripts.aussie_rain import *
from scripts.aussie_rain_process_user_data import *
from scripts.train import *
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree, export_text
%matplotlib inline


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

Завантажую дані в датафрейм

In [3]:
raw_df = pd.read_csv('data/weatherAUS.csv')

In [4]:
raw_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

В датасеті є 145460 рядків, так як RainTomorrow є цільовою змінною, то незапонені рядки нам не потрібні для навчання, а отже їх потрібно видалити

In [6]:
raw_df.dropna(subset=['RainToday', 'RainTomorrow'], inplace=True)

In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140787 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           140787 non-null  object 
 1   Location       140787 non-null  object 
 2   MinTemp        140319 non-null  float64
 3   MaxTemp        140480 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81093 non-null   float64
 6   Sunshine       73982 non-null   float64
 7   WindGustDir    131624 non-null  object 
 8   WindGustSpeed  131682 non-null  float64
 9   WindDir9am     131127 non-null  object 
 10  WindDir3pm     137117 non-null  object 
 11  WindSpeed9am   139732 non-null  float64
 12  WindSpeed3pm   138256 non-null  float64
 13  Humidity9am    139270 non-null  float64
 14  Humidity3pm    137286 non-null  float64
 15  Pressure9am    127044 non-null  float64
 16  Pressure3pm    127018 non-null  float64
 17  Cloud9am       88162 non-null   fl

В даному варіанті вирішення задачі колонка дата не є важливою, так як не використовуються часові ряди і взаємозалежності даних в даті, хоча наприклад місяць може мати значення для ймовірності дощу, тому місяць додам новою фічею

In [8]:
raw_df.RainTomorrow.value_counts()

RainTomorrow
No     109586
Yes     31201
Name: count, dtype: int64

Бачимо незбалансовані класи, а для навчання дерев краще все ж використовувати збалансовані класи

Робимо препроцессинг - а саме - скейлимо числові колонки, крім значення місяця і показника дощу сьогодні, кодуємо категоріальні колонки, балансуємо класи з допомогою SMOTE

In [ ]:
results = preprocess_data(raw_df, True)
print(results['inputs_train'].shape)

Тренуємо модельку

In [11]:
model, train_score, val_score = trainModel(results, max_depth=8)

In [12]:
print(f'Train roc_auc_score {train_score}, \nValidation roc_auc_score {val_score}')

Train roc_auc_score 0.8178468768535839, 
Validation roc_auc_score 0.7678654530522859


In [13]:
joblib.dump(model, 'model/aussie_rain.joblib')

['model/aussie_rain.joblib']

Завантажимо збережену модель і будемо працювати вже з нею

In [14]:
model2 = joblib.load('model/aussie_rain.joblib')
model2.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

Виберемо, які параметри важливі для передбачень, щоб даьи змогу користувачу їх вводити на формі

In [15]:
feature_importances = pd.DataFrame({
    'column':model2.feature_names_in_,
    'importance':model2.feature_importances_
    })

In [16]:
feature_importances.sort_values('importance', ascending=False).head(8)

,column,importance
9,Humidity3pm,0.514202
2,Rainfall,0.120384
4,Sunshine,0.104156
11,Pressure3pm,0.087021
13,Cloud3pm,0.071325
5,WindGustSpeed,0.058399
17,month,0.007516
7,WindSpeed3pm,0.004383


Проведу експеримент - якщо вибрати тільки ці 8 ознак з датафрейму і навчити на них модель - яка буде точність?

In [17]:
valuable_cols = ['Humidity3pm', 'Rainfall', 'Sunshine', 'Pressure3pm', 'Cloud3pm', 'WindGustSpeed', 'Date', 'WindSpeed3pm', 'RainTomorrow']

In [18]:
thin_df = raw_df[valuable_cols]
thin_df.head()

,Humidity3pm,Rainfall,Sunshine,Pressure3pm,Cloud3pm,WindGustSpeed,Date,WindSpeed3pm,RainTomorrow
0,22.0,0.6,NaN,1007.1,NaN,44.0,2008-12-01,24.0,No
1,25.0,0.0,NaN,1007.8,NaN,44.0,2008-12-02,22.0,No
2,30.0,0.0,NaN,1008.7,2.0,46.0,2008-12-03,26.0,No
3,16.0,0.0,NaN,1012.8,NaN,24.0,2008-12-04,9.0,No
4,33.0,1.0,NaN,1006.0,8.0,41.0,2008-12-05,20.0,No


In [ ]:
data_thin = preprocess_data(thin_df, True)

In [20]:
model_thin, train_score_thin, val_score_thin = trainModel(data_thin, max_leaf_nodes=38)

In [21]:
print(f'Train roc_auc_score {train_score_thin}, \nValidation roc_auc_score {val_score_thin}')

Train roc_auc_score 0.8014383811653055, 
Validation roc_auc_score 0.7627520759193358


На валілаційних даних трішки менша точність, проте для такої моделі користувачу не доведеться вводити 23 поля даних, що буде зручнішим в даному випадку, тому я використаю цю модель надалі

In [22]:
joblib.dump(model_thin, 'model/aussie_rain_thin.joblib')

['model/aussie_rain_thin.joblib']

Отримання прогнозу на користувацьких даних

In [23]:
user_cols = valuable_cols.copy()
user_cols.remove('RainTomorrow')


In [ ]:
user_data = pd.DataFrame(np.array([[33, 0.6, 1009, 990, 7, 20, '2024-01-01', 17 ]]), columns = user_cols)

In [24]:
add_data = {
    'scaler' : data_thin['scalerObj'],
    'columns' : user_cols
}
joblib.dump(add_data, 'model/add_data.joblib')

['model/add_data.joblib']

Робимо препроцессинг користувацьких даних

In [277]:
user_data_preprocessed = preprocess_new_data(user_data, data_thin['scalerObj'])

Робимо саме передбачення

In [280]:
model_thin.predict_proba(user_data_preprocessed)

array([[0.55027933, 0.44972067]])